To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [blog post](https://unsloth.ai/blog/r1-reasoning) for guidance on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [35]:
%%capture
# Skip restarting message in Colab
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

!pip install unsloth vllm
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git






In [23]:
from huggingface_hub import login
from google.colab import userdata

hf_token = userdata.get('HF_TOKEN')

login(hf_token)

# 导入wandb库 - Weights & Biases，用于机器学习实验跟踪和可视化
import wandb

wb_token = userdata.get('WB_TOKEN')

wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tune-DeepSeek-R1-Distill-Llama-8B-sql-to-text',    # 设置项目名称 - 这里是用于SQL分析的DeepSeek模型微调项目
    job_type="training",
    anonymous="allow"# 允许匿名访问 # "allow"表示即使没有wandb账号的用户也能查看这个项目
)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


### Unsloth

Use `PatchFastRL` before all functions to patch GRPO and other RL algorithms!

In [49]:
# 从unsloth库中导入FastLanguageModel类
# unsloth是一个优化的语言模型加载和训练库
from unsloth import FastLanguageModel

# 设置模型参数
# 最大序列长度，即模型能处理的最大token数量
max_seq_length = 2048
dtype = None # 数据类型设置为None，让模型自动选择合适的数据类型
load_in_4bit = True # 启用4bit量化加载 # 4bit量化可以显著减少模型内存占用，但可能略微影响模型性能

# 加载预训练模型和分词器
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length = max_seq_length,     # 设置最大序列长度
    dtype = dtype,# 设置数据类型
    load_in_4bit = load_in_4bit,  # 启用4bit量化加载
    token = hf_token, # 使用Hugging Face的访问令牌
)



==((====))==  Unsloth 2025.3.1: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


### Data Prep
<a name="Data"></a>

We directly leverage [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) for data prep and all reward functions. You are free to create your own!

In [50]:
prompt = """Below is SQL query. Think like sql expert and generate a summary of the query which explains the use case of the query. As in
what the query is trying to read from the database in a usecase sense.

### Query:
SELECT
    c.customer_id,
    c.name AS customer_name,
    COUNT(o.order_id) AS total_orders,
    SUM(o.total_amount) AS total_spent,
    AVG(o.total_amount) AS avg_order_value,
    MAX(o.order_date) AS last_order_date
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
LEFT JOIN order_items oi ON o.order_id = oi.order_id
WHERE o.order_date >= '2024-01-01'
GROUP BY c.customer_id, c.name
ORDER BY total_spent DESC
LIMIT 10;

### Response:
"""



In [51]:
# 将模型切换到推理模式
FastLanguageModel.for_inference(model)
# 对输入文本进行分词和编码
# [prompt] - 将prompt放入列表中，因为tokenizer期望批处理输入
# return_tensors="pt" - 返回PyTorch张量格式
# to("cuda") - 将张量移动到GPU上进行计算
inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
# 使用模型生成响应
outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
# 后处理生成的输出
# batch_decode - 将标记ID序列解码回文本
# split("### Response:")[1] - 提取"### Response:"之后的部分，即模型的实际回答
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])



The query is retrieving data about customers who placed orders within a specific timeframe. It provides detailed information for each customer, including their total orders, total spending, average order value, and the date of their last order. The results are grouped by customer and sorted by total spending in descending order, limiting the output to the top 10 customers.

Okay, so I've got this SQL query in front of me, and I need to figure out what it's doing. Let me start by breaking it down piece by piece.

First, the query starts with SELECT, listing several columns from the customers table (c) and some calculated fields. It's selecting customer_id, name, and then some aggregate functions on the orders table (o). 

Looking at the FROM clause, it's joining customers (c) with orders (o) using customer_id as the key. So, it's getting all the orders associated with each customer. Then, there's a LEFT JOIN with order_items (oi) on order_id. Wait, why a LEFT JOIN? Maybe to include cus

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [28]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a SQL expert with advance understanding of SQL queries. You can understand database schema from the query. Think like sql expert and generate a summary of the query which explains the use case of the query. As in
what the query is trying to read from the database in a usecase sense.

### Query:
{}

### Response:
<think>
{}
</think>
{}"""


In [29]:
!pip install datasets
!wget "https://huggingface.co/datasets/b-mc2/sql-create-context/resolve/main/sql_create_context_v4.json"
from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/sql_create_context_v4.json", split="train[0:500]")
# - split: 指定要加载的数据集切片
#   - "train[0:500]" 表示只加载训练集的前500条数据
#   - 这种切片方式可以用于快速实验和调试

EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

#原数据集：输入是英文描述，输出是SQL反转后：输入是SQL(examples["answer"])，输出是英文描述(examples["question"])
def switch_and_format_prompt(examples):
    inputs = examples["answer"] # 使用 answer(SQL) 作为输入
    context = examples["context"]
    outputs = examples["question"] # 使用 question(英文描述) 作为输出
    texts = []
    for input, context, output in zip(inputs, context, outputs):
        text = train_prompt_style.format(input, context, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

# 应用转换
dataset = dataset.map(switch_and_format_prompt, batched = True)





--2025-03-04 06:19:05--  https://huggingface.co/datasets/b-mc2/sql-create-context/resolve/main/sql_create_context_v4.json
Resolving huggingface.co (huggingface.co)... 3.166.152.44, 3.166.152.65, 3.166.152.105, ...
Connecting to huggingface.co (huggingface.co)|3.166.152.44|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/f9/26/f926cb4f40d6cfd6696e03dcc169e95f1b3b9a8e8f965d6040d4bd5f07448faa/280961458ccb74731104ae9af715797a0332327a33a210120a6884085a5c03f0?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27sql_create_context_v4.json%3B+filename%3D%22sql_create_context_v4.json%22%3B&response-content-type=application%2Fjson&Expires=1741072745&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MTA3Mjc0NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy9mOS8yNi9mOTI2Y2I0ZjQwZDZjZmQ2Njk2ZTAzZGNjMTY5ZTk1ZjFiM2I5YThlOGY5NjVkNjA0MGQ0YmQ1ZjA3NDQ4ZmFhLzI4MDk2MTQ1OGNjYjc0NzMxMTA0YWU5YW

In [41]:


model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = 16,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [43]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from unsloth.models.llama import LlamaForCausalLM
from peft import PeftModel

# Monkey patch for_training to avoid deleting non-existent attribute
def for_training(model, use_gradient_checkpointing=True):
    """
    Sets the model to training mode.
    """
    # Check if _unwrapped_old_generate attribute exists before attempting to delete it
    if hasattr(model, "_unwrapped_old_generate"):
        model.generate = model._unwrapped_old_generate
        # Only delete the attribute if it exists
        delattr(model, "_unwrapped_old_generate") # use delattr instead of del

# Patch the original Llama model within the Peft wrapper
# Get the base Llama model from the Peft wrapper
base_model = model.base_model if isinstance(model, PeftModel) else model

# Apply the monkey patch to the base Llama model
LlamaForCausalLM.for_training = for_training
base_model.for_training()

# Now you can proceed with the trainer initialization
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    ## 训练参数配置
    args=TrainingArguments(
        # 批处理相关
        per_device_train_batch_size=2, # 每个设备（GPU）的训练批次大小
        gradient_accumulation_steps=4,# 梯度累积步数，用于模拟更大的批次大小
         # 训练步数和预热
        warmup_steps=5,# 学习率预热步数，逐步增加学习率
        max_steps=60,# 最大训练步数
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(), # 如果不支持 bfloat16，则使用 float16
        bf16=is_bfloat16_supported(),# 如果支持则使用 bfloat16，通常在新型 GPU 上性能更好
        logging_steps=10,# 每10步记录一次日志
        optim="adamw_8bit", # 使用8位精度的 AdamW 优化器
        weight_decay=0.01,# 权重衰减率，用于防止过拟合
        lr_scheduler_type="linear",# 学习率调度器类型，使用线性衰减
        seed=3407,# 随机种子，确保实验可重复性
        output_dir="outputs", # 模型和检查点的输出目录
    ),
)

Converting train dataset to ChatML (num_proc=2):   0%|          | 0/500 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/500 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/500 [00:00<?, ? examples/s]

Truncating train dataset (num_proc=2):   0%|          | 0/500 [00:00<?, ? examples/s]

In [44]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 500 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
10,2.475400
20,0.498300
30,0.362500
40,0.326600
50,0.287000
60,0.289400


In [45]:
# 定义提示模板
# 这个模板包含了指导模型如何理解和解释SQL查询的结构化提示
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a SQL expert with advance understanding of SQL queries. You can understand database schema from the query. Think like sql expert and generate a summary of the query which explains the use case of the query. As in
what the query is trying to read from the database in a usecase sense.

### Query:
{}

### Response:
<think>{}"""

# 定义测试用的SQL查询
# 这是一个复杂的客户分析查询，用于测试模型的理解能力
query1 = """
SELECT
    c.customer_id,
    c.name AS customer_name,
    COUNT(o.order_id) AS total_orders,
    SUM(o.total_amount) AS total_spent,
    AVG(o.total_amount) AS avg_order_value,
    MAX(o.order_date) AS last_order_date
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
LEFT JOIN order_items oi ON o.order_id = oi.order_id
WHERE o.order_date >= '2024-01-01'
GROUP BY c.customer_id, c.name
ORDER BY total_spent DESC
LIMIT 10;
Explain use case of this query.
"""

# 将模型设置为推理模式
FastLanguageModel.for_inference(model)
# 准备输入数据
# 使用提示模板格式化查询，并转换为模型可处理的张量格式
inputs = tokenizer([prompt_style.format(query1, "")], return_tensors="pt").to("cuda")
# 生成响应
outputs = model.generate(
    input_ids=inputs.input_ids, # 输入的标记ID
    attention_mask=inputs.attention_mask, # 注意力掩码，用于处理填充
    max_new_tokens=1200, # 最大生成的新标记数
    use_cache=True,  # 使用缓存以提高生成速度
)
# 解码模型输出
# 使用分词器将输出转换回文本，并提取Response部分
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])
#微调后：解释更加具体和准确 能更好地捕捉查询的完整上下文 例如在示例中，微调后的模型特别指出了"top 10 customers with the highest total spent"这个关键细节，这在微调前的响应中是缺失的



<think>
CREATE TABLE order_items (order_id VARCHAR); CREATE TABLE orders (customer_id VARCHAR, order_id VARCHAR, total_amount VARCHAR, order_date VARCHAR); CREATE TABLE customers (customer_id VARCHAR, name VARCHAR)
</think>
Show customer id, customer name, total orders, total spent, average order value, last order date for top 10 customers by total spent.<｜end▁of▁sentence｜>


Mounted at /content/drive


('/content/drive/MyDrive/deepseek_model/tokenizer_config.json',
 '/content/drive/MyDrive/deepseek_model/special_tokens_map.json',
 '/content/drive/MyDrive/deepseek_model/tokenizer.json')

In [ ]:
# Save to 8bit Q8_0
if True: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "HF_TOKEN")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

In [ ]:
import subprocess
subprocess.Popen(["ollama", "serve"])
import time
time.sleep(3) # Wait for a few seconds for Ollama to load!

In [ ]:
!ollama create unsloth_model -f ./model/Modelfile